In [1]:
from netCDF4 import Dataset
import numpy as np
import math as m
from haversine import haversine as hvs 
from tqdm.notebook import tqdm


### Function and Kernels

Code : Victor Onink (adapted)


In [2]:

def AreaCalc(sizeLon, sizeLat):
    """
    Function that calculates the surface area of grid cells on the Black Sea.
    
    param BinsLon: the grid cell size in longitudinal direction
    param BinsLat: the grid cell size in latitudinal direction
    returns: matrix with the surface area (in km^2) of each grid cell
    """
    #Black Sea area :calculate the approx size in km in zonal/merdional direction of the BS polygon
    lon11 = (48,27)  #max lat, min lon
    lon12 = (48,42)
    d_lonA = hvs(lon11,lon12)

    lon21 = (41,27)
    lon22 = (41,42)
    d_lonB = hvs(lon21,lon22)

    d_lon = np.mean([d_lonA,d_lonB])
    d_lon
    
    lat11 = (41,27)
    lat21 = (48,27)
    d_latA = hvs(lat11,lat21)

    lat12 = (41,42)
    lat22 = (48,42)
    d_latB = hvs(lat12,lat22)

    d_lat = np.mean([d_latA,d_latB])
    d_lat
    
    
    #Area
    radians = np.pi/180. 
    lon_bins = np.linspace(26.5,42, sizeLon+1) # the bins in longitudinal direction
    lat_bins = np.linspace(41,48, sizeLat+1) # the bins in latitudinal direction
    Area = np.array([[radians*(lon_bins[i+1]-lon_bins[i])*
                      (np.sin(radians*lat_bins[j+1]) - np.sin(radians*lat_bins[j])) 
                      for j in range(len(lat_bins)-1)]
                      for i in range(len(lon_bins)-1)])
    
    Area = d_lon*d_lat*Area     #km^2
    return Area

def HistogramFunction(binslon,binslat,londata,latdata):
    londata,latdata=londata.reshape(np.size(londata)),latdata.reshape(np.size(latdata))
    density=np.zeros((len(binsLon),len(binsLat)))
    
    for i in range(np.array(londata).shape[0]):
        density[np.argmin(np.abs(londata[i]-binsLon)),np.argmin(np.abs(latdata[i]-binsLat))]+=1
        
    #Normalize it by area
    area=AreaCalc(len(binsLat),len(binsLon)).T
    density/=area
    density[density==0]=np.nan
    return density

def load_data(ddir,f):
    dataset=Dataset(ddir+f)
    lat=dataset.variables['lat'][:]
    lon=dataset.variables['lon'][:]
    time=dataset.variables['time'][:]
    beached=dataset.variables['beached'][:]
    lon[lon>180]-=360
    lon[lon<-180]+=360
    
    return lon, lat, time, beached

### Load Data

In [4]:

ddir = r'C:\Users\Administrateur\Documents\CP_18-20\Thesis\Parcels'

files =[
        '\Rivers\RUN_SAKARYA8_365d.nc',
        '\Rivers\RUN_KIZILIRMAK8_365d.nc',
        '\Rivers\RUN_KODORI8_365d.nc',
        '\Rivers\RUN_YESILIMARK8_365d.nc',
        '\Rivers\RUN_BZYB8_365d.nc', 
        '\Rivers\RUN_DON8_365d.nc'
    
        ]

lon,lat,time,beached= [[] for i in range(4)]

for f in files :
    lo,la,t,b= load_data(ddir,f)
    lon.append(lo)
    lat.append(la)
    time.append(t)
    beached.append(b)

C:\Anaconda3\envs\py3_parcels\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in greater
C:\Anaconda3\envs\py3_parcels\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in less


### Calculate Density

In [5]:
# Black Sea coordinates
minlon = 27
maxlon = 42
minlat = 41
maxlat = 48


# All data: mean over 1 year
lonLast=lon[0][:,:] 
latLast=lat[0][:,:]
Time=time[0][:,:]

binsLon=np.arange(minlon,maxlon,0.1)
binsLat=np.arange(minlat,maxlat,0.1)
density=np.zeros((lonLast.shape[1],len(binsLon),len(binsLat)))

for i in tqdm(range(Time.shape[1])):
    density[i,:,:]=HistogramFunction(binsLon,binsLat,lonLast[:,i],latLast[:,i])

saveFiles=str(ddir)+r'\Rivers\Density_8\Sakarya8'
density.dump(saveFiles)
Time.dump(saveFiles+'_Time')